In [1]:
print('started')

started


In [2]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

2.1.1
1.26.0


In [3]:
file_path = 'metadata.tsv'
df = pd.read_csv(file_path, sep='\t')

/var/folders/2n/gzd0gjnd7zg2s5l8kbwzsr7h0000gn/T/ipykernel_12967/1980348128.py:2: DtypeWarning: Columns (4,5,10,29,35,36,52,57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t')


In [4]:
df.head()

,strain,virus,gisaid_epi_isl,genbank_accession,genbank_accession_rev,sra_accession,date,region,country,division,...,QC_frame_shifts,QC_stop_codons,QC_overall_score,QC_overall_status,frame_shifts,deletions,insertions,substitutions,aaSubstitutions,clock_deviation
0,?,ncov,?,OX592216,OX592216.1,ERR5511452,?,Europe,United Kingdom,?,...,good,good,2268.049660,bad,NaN,NaN,27400:GTCATCCAATTT,"G23401T,G25440T,C25614T,C26801G,G29645T","ORF3a:K16N,ORF7a:I3F,S:Q613H",?
1,ARG/210316-1/2021,ncov,?,OL989073,OL989073.1,NaN,2021-03-15,South America,Argentina,Argentina,...,good,good,14.062500,good,NaN,NaN,NaN,"C66T,C241T,C3037T,A4926G,A5574G,C7564A,C8139T,...","N:R203K,N:G204R,ORF1a:D1554G,ORF1a:Y1770C,ORF1...",2.0
2,ARG/210524-2/2021,ncov,?,OL989092,OL989092.1,NaN,2021-05-17,South America,Argentina,Argentina,...,good,good,4.340278,good,NaN,NaN,NaN,"A22G,C241T,C913T,C3037T,C3177T,C3267T,C5388A,C...","N:D3L,N:R203K,N:G204R,N:S235F,N:A308S,ORF1a:P9...",3.0
3,ARG/210610-225/2021,ncov,?,OL989093,OL989093.1,NaN,2021-06-07,South America,Argentina,Argentina,...,good,good,50.173611,mediocre,NaN,NaN,NaN,"C241T,C913T,A1967G,C3037T,C3177T,C3267T,C5388A...","N:D3L,N:P151L,N:R203K,N:G204R,N:S235F,ORF1a:T5...",0.0
4,ARG/210613-39/2021,ncov,?,OL989084,OL989084.1,NaN,2021-06-07,South America,Argentina,Argentina,...,good,good,6.250000,good,NaN,NaN,NaN,"C241T,T733C,A2596G,C2749T,C3037T,C3828T,A5648C...","M:H125Y,N:P80R,N:R203K,N:G204R,ORF1a:S1188L,OR...",0.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9318359 entries, 0 to 9318358
Data columns (total 58 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   strain                  object 
 1   virus                   object 
 2   gisaid_epi_isl          object 
 3   genbank_accession       object 
 4   genbank_accession_rev   object 
 5   sra_accession           object 
 6   date                    object 
 7   region                  object 
 8   country                 object 
 9   division                object 
 10  location                object 
 11  region_exposure         object 
 12  country_exposure        object 
 13  division_exposure       object 
 14  segment                 object 
 15  length                  int64  
 16  host                    object 
 17  age                     object 
 18  sex                     object 
 19  Nextstrain_clade        object 
 20  pango_lineage           object 
 21  GISAID_clade            object 

In [7]:
def clean_metadata(df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies a series of cleaning and filtering steps to the raw SARS-CoV-2 metadata DataFrame.

    Args:
        df (pd.DataFrame): The raw DataFrame loaded from the metadata file.

    Returns:
        pd.DataFrame: A cleaned and filtered DataFrame.
    """
    print(f"Starting cleaning process with {df.shape[0]:,} rows.")
    
    # Create a copy to avoid modifying the original DataFrame in memory
    df_cleaned = df.copy()

    # --- Filter 1: Remove rows with no mutation data ---
    # Rows where substitutions, deletions, AND insertions are all empty are dropped.
    # We check for both NaN and empty strings for robustness.
    mutation_cols = ['substitutions', 'deletions', 'insertions']
    # Ensure all required columns are present before filtering
    if all(col in df_cleaned.columns for col in mutation_cols):
        is_empty_mask = (df_cleaned['substitutions'].isna() | (df_cleaned['substitutions'] == '')) & \
                        (df_cleaned['deletions'].isna() | (df_cleaned['deletions'] == '')) & \
                        (df_cleaned['insertions'].isna() | (df_cleaned['insertions'] == ''))
        
        # We keep the rows where the 'is_empty_mask' is False
        df_cleaned = df_cleaned[~is_empty_mask]
        print(f"-> {df_cleaned.shape[0]:,} rows remaining after removing entries with no mutation data.")
    else:
        print(f"Warning: One or more mutation columns not found. Skipping mutation data filter.")


    # --- Filter 2: Coverage ---
    # Keep rows where coverage is >= 0.99. This will also drop rows where coverage is NaN.
    if 'coverage' in df_cleaned.columns:
        df_cleaned = df_cleaned[df_cleaned['coverage'] >= 0.99]
        print(f"-> {df_cleaned.shape[0]:,} rows remaining after filtering for coverage >= 0.99.")
    else:
        print("Warning: 'coverage' column not found. Skipping coverage filter.")


    # --- Filter 3: Missing Data ---
    # Keep rows where missing_data is < 0.02. This also drops rows where missing_data is NaN.
    if 'missing_data' in df_cleaned.columns:
        df_cleaned = df_cleaned[df_cleaned['missing_data'] < 0.02]
        print(f"-> {df_cleaned.shape[0]:,} rows remaining after filtering for missing_data < 0.02.")
    else:
        print("Warning: 'missing_data' column not found. Skipping missing_data filter.")


    # --- Filter 4: Virus Type ---
    # Keep only rows where the virus is 'ncov'. Case-sensitive.
    if 'virus' in df_cleaned.columns:
        df_cleaned = df_cleaned[df_cleaned['virus'] == 'ncov']
        print(f"-> {df_cleaned.shape[0]:,} rows remaining after filtering for virus == 'ncov'.")
    else:
        print("Warning: 'virus' column not found. Skipping virus filter.")


    # --- Filter 5: Essential Non-Null Columns ---
    # Drop rows if any of these specific columns are null/empty.
    essential_cols = ['virus', 'length', 'date_submitted']
    if all(col in df_cleaned.columns for col in essential_cols):
        df_cleaned.dropna(subset=essential_cols, inplace=True)
        print(f"-> {df_cleaned.shape[0]:,} rows remaining after dropping rows with null essential columns (virus, length, date_submitted).")
    else:
        print(f"Warning: One or more essential columns not found. Skipping essential columns filter.")
        

    # --- Filter 6: QC Status Columns ---
    # Keep only rows that pass specific QC checks.
    qc_filters = {
        'QC_overall_status': 'bad',
        'QC_missing_data': 'good',
        'QC_frame_shifts': 'good',
        'QC_stop_codons': 'good',
        'QC_mixed_sites': 'good'
    }
    
    for col, status in qc_filters.items():
        if col in df_cleaned.columns:
            initial_rows = df_cleaned.shape[0]
            if status == 'bad':
                # For 'bad', we keep rows that are NOT 'bad'. This keeps 'good', 'mediocre', and also NaNs.
                # To be strict, we'll explicitly drop NaNs in this column as well.
                df_cleaned = df_cleaned[(df_cleaned[col] != 'bad') & (df_cleaned[col].notna())]
            else:
                # For 'good', we keep only rows that ARE 'good'. This automatically handles NaNs.
                df_cleaned = df_cleaned[df_cleaned[col] == 'good']
            
            print(f"-> {df_cleaned.shape[0]:,} rows remaining after filtering for '{col}' quality check ({initial_rows - df_cleaned.shape[0]:,} rows removed).")
        else:
            print(f"Warning: QC column '{col}' not found. Skipping this QC filter.")

    print(f"\nCleaning complete. Final dataset has {df_cleaned.shape[0]:,} rows.")
    return df_cleaned

In [8]:
cleaned_df = clean_metadata(df)

Starting cleaning process with 9,318,359 rows.
-> 9,317,110 rows remaining after removing entries with no mutation data.
-> 5,486,538 rows remaining after filtering for coverage >= 0.99.
-> 1,782,528 rows remaining after filtering for missing_data < 0.02.
-> 1,782,528 rows remaining after filtering for virus == 'ncov'.
-> 1,782,528 rows remaining after dropping rows with null essential columns (virus, length, date_submitted).
-> 1,745,951 rows remaining after filtering for 'QC_overall_status' quality check (36,577 rows removed).
-> 1,745,951 rows remaining after filtering for 'QC_missing_data' quality check (0 rows removed).
-> 1,741,700 rows remaining after filtering for 'QC_frame_shifts' quality check (4,251 rows removed).
-> 1,740,840 rows remaining after filtering for 'QC_stop_codons' quality check (860 rows removed).
-> 1,705,483 rows remaining after filtering for 'QC_mixed_sites' quality check (35,357 rows removed).

Cleaning complete. Final dataset has 1,705,483 rows.


In [9]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1705483 entries, 7 to 9318352
Data columns (total 58 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   strain                  object 
 1   virus                   object 
 2   gisaid_epi_isl          object 
 3   genbank_accession       object 
 4   genbank_accession_rev   object 
 5   sra_accession           object 
 6   date                    object 
 7   region                  object 
 8   country                 object 
 9   division                object 
 10  location                object 
 11  region_exposure         object 
 12  country_exposure        object 
 13  division_exposure       object 
 14  segment                 object 
 15  length                  int64  
 16  host                    object 
 17  age                     object 
 18  sex                     object 
 19  Nextstrain_clade        object 
 20  pango_lineage           object 
 21  GISAID_clade            object 
 22 

In [10]:
cleaned_df.head()

,strain,virus,gisaid_epi_isl,genbank_accession,genbank_accession_rev,sra_accession,date,region,country,division,...,QC_frame_shifts,QC_stop_codons,QC_overall_score,QC_overall_status,frame_shifts,deletions,insertions,substitutions,aaSubstitutions,clock_deviation
7,ARG/Cordoba-1006-155/2020,ncov,?,MW553298,MW553298.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,0.000000,good,NaN,NaN,NaN,"C241T,C3037T,C3261T,C9711T,C14408T,C17012T,A20...","ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0
8,ARG/Cordoba-1083-6/2020,ncov,?,MW553296,MW553296.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,0.173611,good,NaN,NaN,NaN,"C241T,C3037T,C3261T,C9711T,C14408T,C17012T,A20...","ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",5.0
10,ARG/Cordoba-11301-61/2020,ncov,?,MW553314,MW553314.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,0.000000,good,NaN,NaN,NaN,"C241T,C3037T,C3261T,C9711T,C14408T,C17012T,A20...","ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0
11,ARG/Cordoba-11419-61/2020,ncov,?,MW553297,MW553297.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,0.000000,good,NaN,NaN,NaN,"C241T,C3037T,C3261T,C9711T,C14408T,C17012T,A20...","ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0
12,ARG/Cordoba-11499-61/2020,ncov,?,MW553316,MW553316.1,NaN,2020-06-04,South America,Argentina,Argentina,...,good,good,0.173611,good,NaN,25416-25424,NaN,"C241T,C3037T,C3261T,C9711T,C14408T,C17012T,A20...","ORF1a:T999I,ORF1a:S3149F,ORF1b:P314L,ORF1b:S11...",4.0


In [11]:
import re

def _transform_deletion_part(part: str) -> str:
    """Transforms a single deletion entry (e.g., '11288-11297' -> '11288_11297')."""
    return part.replace('-', '_')

def _transform_insertion_part(part: str) -> str:
    """Transforms a single insertion entry (e.g., '28263:A' -> '28263_.|A')."""
    return part.replace(':', '_.|')

def _transform_substitution_part(part: str) -> str:
    """
    Transforms a single substitution entry using regex (e.g., 'C241T' -> '241_C|T').
    Captures the leading base(s), the number, and the trailing base(s).
    """
    # Regex: (Group 1: one or more non-digits) (Group 2: one or more digits) (Group 3: one or more non-digits)
    try:
        match = re.fullmatch(r'([A-Z-]+)(\d+)([A-Z-]+)', part, re.IGNORECASE)
        if match:
            base1, number, base2 = match.groups()
            return f"{number}_{base1}|{base2}"
    except Exception as e:
        print(f"Error processing part '{part}': {e}")    
    
    # Return the original part if it doesn't match the expected format
    return part

In [14]:
def process_mutations(df: pd.DataFrame) -> pd.DataFrame:
    """
    Processes the 'substitutions', 'deletions', and 'insertions' columns
    of a DataFrame, transforms them to a desired format, and combines them
    into a single 'mutations' column.

    Args:
        df (pd.DataFrame): The input DataFrame containing the mutation columns.

    Returns:
        pd.DataFrame: The DataFrame with the new 'mutations' column.
    """
    print("Starting transformation of mutation columns...")
    
    # Create a copy to avoid modifying the original DataFrame passed to the function
    df_processed = df.copy()

    # Define a generic transformation function that applies a specific part_transformer
    def transform_column(column_series: pd.Series, part_transformer_func) -> pd.Series:
        """
        Takes a Series (a column), splits each cell by comma, applies a
        transformation to each part, and joins them back. Handles NaNs.
        """
        # Fill NaN with empty string to prevent errors
        # Convert to string to ensure .str accessor works
        series_str = column_series.fillna('').astype(str)
        
        # Apply the transformation
        def process_cell(cell_value: str):
            if not cell_value:
                return ""
            parts = cell_value.split(',')
            transformed_parts = [part_transformer_func(part.strip()) for part in parts]
            return ",".join(transformed_parts)
            
        return series_str.apply(process_cell)

    # --- Apply transformations to each column ---
    
    # Process Substitutions
    if 'substitutions' in df_processed.columns:
        print("Processing 'substitutions'...")
        df_processed['substitutions_transformed'] = transform_column(
            df_processed['substitutions'], _transform_substitution_part
        )
    else:
        print("Warning: 'substitutions' column not found. Skipping.")
        df_processed['substitutions_transformed'] = ''

    # Process Deletions
    if 'deletions' in df_processed.columns:
        print("Processing 'deletions'...")
        df_processed['deletions_transformed'] = transform_column(
            df_processed['deletions'], _transform_deletion_part
        )
    else:
        print("Warning: 'deletions' column not found. Skipping.")
        df_processed['deletions_transformed'] = ''
        
    # Process Insertions
    if 'insertions' in df_processed.columns:
        print("Processing 'insertions'...")
        df_processed['insertions_transformed'] = transform_column(
            df_processed['insertions'], _transform_insertion_part
        )
    else:
        print("Warning: 'insertions' column not found. Skipping.")
        df_processed['insertions_transformed'] = ''

    # --- Combine into final 'mutations' column ---
    print("Combining transformed columns into 'mutations' column...")
    
    # Select the transformed columns to work with
    transformed_cols = [
        'substitutions_transformed', 
        'deletions_transformed', 
        'insertions_transformed'
    ]
    
    # Apply a function row-wise (axis=1) to the DataFrame subset.
    # This correctly joins the mutation strings from each row.
    df_processed['mutations'] = df_processed[transformed_cols].apply(
        lambda row: ",".join(filter(None, row)), # filter(None, row) removes empty strings
        axis=1
    )
    
    print("Mutation processing complete.")
    return df_processed

In [15]:
df_with_mutations = process_mutations(cleaned_df)

Starting transformation of mutation columns...
Processing 'substitutions'...
Processing 'deletions'...
Processing 'insertions'...
Combining transformed columns into 'mutations' column...
Mutation processing complete.


In [ ]:
display(df_with_mutations[['substitutions_transformed', 'deletions_transformed', 'insertions_transformed', 'mutations']])

,substitutions_transformed,deletions_transformed,insertions_transformed,mutations
7,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1...",,,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
8,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1...",,,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
10,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1...",,,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
11,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1...",,,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."
12,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1...",25416_25424,,"241_C|T,3037_C|T,3261_C|T,9711_C|T,14408_C|T,1..."


In [17]:
# Define the regex patterns for each desired format
# Handles cases like '241_C|T'
SUBSTITUTION_PATTERN = re.compile(r"^\d+_[A-Z-]+\|[A-Z-]+$", re.IGNORECASE)
# Handles '11288' or '11288_11297'
DELETION_PATTERN = re.compile(r"^\d+(_\d+)?$")
# Handles '28263_.|A' or '28263_.|ACGT'
INSERTION_PATTERN = re.compile(r"^\d+_\.\|[A-Z-]+$", re.IGNORECASE)

def validate_column_format(df: pd.DataFrame, column_name: str, pattern: re.Pattern) -> bool:
    """
    Validates that every comma-separated part in a DataFrame column matches a regex pattern.

    Returns True if all parts in all rows are valid, otherwise prints errors and returns False.
    """
    print(f"Validating format for column: '{column_name}'...")
    # Select non-empty rows for the column to validate
    series_to_check = df[df[column_name] != ''][column_name]
    
    # If the series is empty after filtering, there's nothing to validate
    if series_to_check.empty:
        print("  Column is empty or has no non-empty values. Validation skipped/passed.")
        return True

    is_valid = True
    # .str.split(',') returns a Series of lists
    for index, mutation_list in series_to_check.str.split(',').items():
        for mutation in mutation_list:
            if not pattern.fullmatch(mutation.strip()):
                print(f"  ERROR: Invalid format found in row index {index}, column '{column_name}'.")
                print(f"    -> Problematic value: '{mutation}'")
                is_valid = False
    
    if is_valid:
        print(f"  SUCCESS: All entries in '{column_name}' are in the correct format.")
    
    return is_valid

# Assuming 'df_with_mutations' is your DataFrame and it now contains the _transformed columns
try:
    # Validate each column using assert
    assert validate_column_format(df_with_mutations, 'substitutions_transformed', SUBSTITUTION_PATTERN), "Substitution format validation failed!"
    assert validate_column_format(df_with_mutations, 'deletions_transformed', DELETION_PATTERN), "Deletion format validation failed!"
    assert validate_column_format(df_with_mutations, 'insertions_transformed', INSERTION_PATTERN), "Insertion format validation failed!"
    
    print("\nAll format assertions passed successfully!")

except AssertionError as e:
    print(f"\nVALIDATION FAILED: {e}")

Validating format for column: 'substitutions_transformed'...
  SUCCESS: All entries in 'substitutions_transformed' are in the correct format.
Validating format for column: 'deletions_transformed'...
  SUCCESS: All entries in 'deletions_transformed' are in the correct format.
Validating format for column: 'insertions_transformed'...
  SUCCESS: All entries in 'insertions_transformed' are in the correct format.

All format assertions passed successfully!


In [18]:
# Drop the intermediate transformed columns
df_final = df_with_mutations.drop(
    columns=['substitutions_transformed', 'deletions_transformed', 'insertions_transformed', 'substitutions', 'deletions', 'insertions']
)

In [19]:
df_final.to_csv('metadata_cleaned.tsv', sep='\t', index=False)